Import libraries and upload data from nflverse github

In [1]:
#Import Pandas
import pandas as pd

#Import requests
import requests

#Import beautifulsoup
from bs4 import BeautifulSoup

/Users/jonathan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


All data used in this notebook is from Pro-football-reference.com

In [2]:
# List of URLs you want to scrape
urls = [
    'https://www.pro-football-reference.com/years/2024/games.htm',
    'https://www.pro-football-reference.com/years/2023/games.htm',
    'https://www.pro-football-reference.com/years/2022/games.htm',
    'https://www.pro-football-reference.com/years/2021/games.htm',
    'https://www.pro-football-reference.com/years/2020/games.htm',
    'https://www.pro-football-reference.com/years/2019/games.htm',
    'https://www.pro-football-reference.com/years/2018/games.htm',
    'https://www.pro-football-reference.com/years/2017/games.htm'
]

# Function to scrape data from a single URL and return a DataFrame
def scrape_games_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the main games table
    table = soup.find('table', id='games')

    # Initialize a list to store data
    data = []

    # Extract data from table rows
    for row in table.find_all('tr')[1:]:  # Skip header row
        cols = row.find_all(['th', 'td'])
        if len(cols) > 0:
            week = cols[0].text.strip()
            day = cols[1].text.strip()
            date = cols[2].text.strip()
            time = cols[3].text.strip()
            winner = cols[4].text.strip()
            awaygame = cols[5].text.strip()
            loser = cols[6].text.strip()
            pts_w = cols[8].text.strip()
            pts_l = cols[9].text.strip()
            yds_w = cols[10].text.strip()
            to_w = cols[11].text.strip()
            yds_l = cols[12].text.strip()
            to_l = cols[13].text.strip()

            data.append([week, day, date, time, winner, awaygame, loser, pts_w, pts_l, yds_w, to_w, yds_l, to_l])

    # Create DataFrame
    games_df = pd.DataFrame(data, columns=['Week', 'Day', 'Date', 'Time', 'Winner', 'Awaygame', 'Loser', 
                                           'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL'])
    return games_df

# List to store DataFrames
all_games_data = []

# Loop over the URLs and scrape data
for url in urls:
    df = scrape_games_data(url)
    all_games_data.append(df)

# Optionally, concatenate all the DataFrames into one large DataFrame
combined_games_data = pd.concat(all_games_data, ignore_index=True)

# Print the first few rows of the combined DataFrame
print(combined_games_data.head())
print(combined_games_data.info())
print(combined_games_data.shape)

  Week  Day        Date    Time               Winner Awaygame  \
0    1  Thu  2024-09-05  8:20PM   Kansas City Chiefs            
1    1  Fri  2024-09-06  8:15PM  Philadelphia Eagles            
2    1  Sun  2024-09-08  1:00PM  Pittsburgh Steelers        @   
3    1  Sun  2024-09-08  1:00PM        Buffalo Bills            
4    1  Sun  2024-09-08  1:00PM   New Orleans Saints            

               Loser PtsW PtsL YdsW TOW YdsL TOL  
0   Baltimore Ravens   27   20  353   1  452   1  
1  Green Bay Packers   34   29  410   3  414   1  
2    Atlanta Falcons   18   10  270   0  226   3  
3  Arizona Cardinals   34   28  352   1  270   1  
4  Carolina Panthers   47   10  379   1  193   3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2349 entries, 0 to 2348
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Week      2349 non-null   object
 1   Day       2349 non-null   object
 2   Date      2349 non-null   object
 3   T

In [3]:
#Data describe
print(combined_games_data.describe())

        Week   Day  Date    Time      Winner Awaygame      Loser  PtsW  PtsL  \
count   2349  2349  2349    2349        2349     2349       2349  2349  2349   
unique    24     9   478      34          37        3         37    56    45   
top     Week   Sun  Date  1:00PM  Winner/tie           Loser/tie    27    17   
freq     132  1821   132    1099         132     1347        132   182   222   

        YdsW   TOW  YdsL   TOL  
count   2349  2349  2349  2349  
unique   377     9   404    10  
top     YdsW     0  YdsL     1  
freq     132   920   132   687  


In [4]:
#Check awaygame column
print(combined_games_data["Awaygame"].unique())

['' '@' 'N']


In [5]:
#Check if ties are indicated correctly with known tie games, will use Detroit Lions vs Pittsburgh Steelers in week 10 of 2021 season
combined_games_data[combined_games_data["Date"].isin(["2021-11-14"])]

,Week,Day,Date,Time,Winner,Awaygame,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL
1054,10,Sun,2021-11-14,1:00PM,Dallas Cowboys,,Atlanta Falcons,43,3,431,1,214,3
1055,10,Sun,2021-11-14,1:00PM,Buffalo Bills,@,New York Jets,45,17,489,2,366,5
1056,10,Sun,2021-11-14,1:00PM,Pittsburgh Steelers,,Detroit Lions,16,16,387,3,306,0
1057,10,Sun,2021-11-14,1:00PM,Tennessee Titans,,New Orleans Saints,23,21,264,0,373,1
1058,10,Sun,2021-11-14,1:00PM,Washington Football Team,,Tampa Bay Buccaneers,29,19,320,1,273,2
1059,10,Sun,2021-11-14,1:00PM,New England Patriots,,Cleveland Browns,45,7,452,0,217,1
1060,10,Sun,2021-11-14,1:00PM,Indianapolis Colts,,Jacksonville Jaguars,23,17,295,0,331,1
1061,10,Sun,2021-11-14,4:05PM,Minnesota Vikings,@,Los Angeles Chargers,27,20,381,1,253,1
1062,10,Sun,2021-11-14,4:05PM,Carolina Panthers,@,Arizona Cardinals,34,10,341,2,169,2
1063,10,Sun,2021-11-14,4:25PM,Philadelphia Eagles,@,Denver Broncos,30,13,386,1,308,1


In [6]:
#Adding tie column
combined_games_data['Tie'] = (combined_games_data['PtsW'] == combined_games_data['PtsL']).astype(int)

In [7]:
#Look at tie games
combined_games_data[combined_games_data["Tie"]==1]

,Week,Day,Date,Time,Winner,Awaygame,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Tie
16,Week,Day,Date,Time,Winner/tie,,Loser/tie,Pts,Pts,YdsW,TOW,YdsL,TOL,1
33,Week,Day,Date,Time,Winner/tie,,Loser/tie,Pts,Pts,YdsW,TOW,YdsL,TOL,1
50,Week,Day,Date,Time,Winner/tie,,Loser/tie,Pts,Pts,YdsW,TOW,YdsL,TOL,1
67,Week,Day,Date,Time,Winner/tie,,Loser/tie,Pts,Pts,YdsW,TOW,YdsL,TOL,1
82,Week,Day,Date,Time,Winner/tie,,Loser/tie,Pts,Pts,YdsW,TOW,YdsL,TOL,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2269,Week,Day,Date,Time,Winner/tie,,Loser/tie,Pts,Pts,YdsW,TOW,YdsL,TOL,1
2286,Week,Day,Date,Time,Winner/tie,,Loser/tie,Pts,Pts,YdsW,TOW,YdsL,TOL,1
2303,Week,Day,Date,Time,Winner/tie,,Loser/tie,Pts,Pts,YdsW,TOW,YdsL,TOL,1
2320,Week,Day,Date,Time,Winner/tie,,Loser/tie,Pts,Pts,YdsW,TOW,YdsL,TOL,1


In [8]:
#Look for non-dates in date column
print(combined_games_data["Date"].unique())

['2024-09-05' '2024-09-06' '2024-09-08' '2024-09-09' 'Date' '2024-09-12'
 '2024-09-15' '2024-09-16' '2024-09-19' '2024-09-22' '2024-09-23'
 '2024-09-26' '2024-09-29' '2024-09-30' '2024-10-03' '2024-10-06'
 '2024-10-07' '2024-10-10' '2024-10-13' '2024-10-14' '2024-10-17'
 '2024-10-20' '2024-10-21' '2024-10-24' '2024-10-27' '2024-10-28'
 '2024-10-31' '2024-11-03' '2024-11-04' '2024-11-07' '2024-11-10'
 '2024-11-11' '2024-11-14' '2024-11-17' '2024-11-18' '2024-11-21'
 '2024-11-24' '2024-11-25' '2024-11-28' '2024-11-29' '2024-12-01'
 '2024-12-02' '2024-12-05' '2024-12-08' '2024-12-09' '2024-12-12'
 '2024-12-15' '2024-12-16' '2024-12-19' '2024-12-21' '2024-12-22'
 '2024-12-23' '2024-12-25' '2024-12-26' '2024-12-28' '2024-12-29'
 '2024-12-30' '2025-01-04' '2025-01-05' 'Playoffs' '2025-01-11'
 '2025-01-12' '2025-01-13' '2025-01-18' '2025-01-19' '2025-01-26'
 '2025-02-09' '2023-09-07' '2023-09-10' '2023-09-11' '2023-09-14'
 '2023-09-17' '2023-09-18' '2023-09-21' '2023-09-24' '2023-09-25'
 '202

In [9]:
#Look at rows containing non dates in date column
searchvalues = ["Date", "Playoffs"]
print(combined_games_data[combined_games_data["Date"].isin(searchvalues)])

      Week  Day      Date  Time      Winner Awaygame      Loser PtsW PtsL  \
16    Week  Day      Date  Time  Winner/tie           Loser/tie  Pts  Pts   
33    Week  Day      Date  Time  Winner/tie           Loser/tie  Pts  Pts   
50    Week  Day      Date  Time  Winner/tie           Loser/tie  Pts  Pts   
67    Week  Day      Date  Time  Winner/tie           Loser/tie  Pts  Pts   
82    Week  Day      Date  Time  Winner/tie           Loser/tie  Pts  Pts   
...    ...  ...       ...   ...         ...      ...        ...  ...  ...   
2269  Week  Day      Date  Time  Winner/tie           Loser/tie  Pts  Pts   
2286  Week  Day      Date  Time  Winner/tie           Loser/tie  Pts  Pts   
2303  Week  Day      Date  Time  Winner/tie           Loser/tie  Pts  Pts   
2320  Week  Day      Date  Time  Winner/tie           Loser/tie  Pts  Pts   
2337             Playoffs                                                   

      YdsW  TOW  YdsL  TOL  Tie  
16    YdsW  TOW  YdsL  TOL    1  
33    Y

In [10]:
# Convert 'Date' column to datetime, with coercion of errors (invalid dates become NaT)
combined_games_data["Date"] = pd.to_datetime(combined_games_data["Date"], errors='coerce')

# Drop rows with NaT (invalid dates) if needed
combined_games_data = combined_games_data.dropna(subset=["Date"])

# Set 'Date' column as index
combined_games_data.set_index("Date", inplace=True)

# Print the DataFrame
print(combined_games_data)

                 Week  Day    Time                Winner Awaygame  \
Date                                                                
2024-09-05          1  Thu  8:20PM    Kansas City Chiefs            
2024-09-06          1  Fri  8:15PM   Philadelphia Eagles            
2024-09-08          1  Sun  1:00PM   Pittsburgh Steelers        @   
2024-09-08          1  Sun  1:00PM         Buffalo Bills            
2024-09-08          1  Sun  1:00PM    New Orleans Saints            
...               ...  ...     ...                   ...      ...   
2018-01-14   Division  Sun  1:05PM  Jacksonville Jaguars        @   
2018-01-14   Division  Sun  4:40PM     Minnesota Vikings            
2018-01-21  ConfChamp  Sun  3:05PM  New England Patriots            
2018-01-21  ConfChamp  Sun  6:40PM   Philadelphia Eagles            
2018-02-04  SuperBowl  Sun  6:30PM   Philadelphia Eagles        N   

                           Loser PtsW PtsL YdsW TOW YdsL TOL  Tie  
Date                              

In [11]:
#Look at tie games again
combined_games_data[combined_games_data["Tie"]==1]

,Week,Day,Time,Winner,Awaygame,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Tie
Date,,,,,,,,,,,,,
2022-09-11,1,Sun,1:00PM,Houston Texans,,Indianapolis Colts,20,20,299,1,517,2,1
2022-12-04,13,Sun,1:00PM,Washington Commanders,@,New York Giants,20,20,411,1,316,1,1
2021-11-14,10,Sun,1:00PM,Pittsburgh Steelers,,Detroit Lions,16,16,387,3,306,0,1
2020-09-27,3,Sun,1:00PM,Philadelphia Eagles,,Cincinnati Bengals,23,23,378,2,304,0,1
2019-09-08,1,Sun,4:25PM,Detroit Lions,@,Arizona Cardinals,27,27,477,2,387,1,1
2018-09-09,1,Sun,1:00PM,Pittsburgh Steelers,@,Cleveland Browns,21,21,472,6,327,1,1
2018-09-16,2,Sun,1:00PM,Minnesota Vikings,@,Green Bay Packers,29,29,480,1,351,0,1


Superbowl data will be available after this weekend, should solve the tie value there

In [12]:
#Check for missing values
print(combined_games_data.isnull().sum().sort_values(ascending=False))

Week        0
Day         0
Time        0
Winner      0
Awaygame    0
Loser       0
PtsW        0
PtsL        0
YdsW        0
TOW         0
YdsL        0
TOL         0
Tie         0
dtype: int64


In [13]:
combined_games_data.dtypes

Week        object
Day         object
Time        object
Winner      object
Awaygame    object
Loser       object
PtsW        object
PtsL        object
YdsW        object
TOW         object
YdsL        object
TOL         object
Tie          int64
dtype: object

In [14]:
#converting numeric data
combined_games_data['PtsW'] = pd.to_numeric(combined_games_data['PtsW'], errors='coerce')
combined_games_data['PtsL'] = pd.to_numeric(combined_games_data['PtsL'], errors='coerce')
combined_games_data['YdsW'] = pd.to_numeric(combined_games_data['YdsW'], errors='coerce')
combined_games_data['YdsL'] = pd.to_numeric(combined_games_data['YdsL'], errors='coerce')
combined_games_data['TOW'] = pd.to_numeric(combined_games_data['TOW'], errors='coerce')
combined_games_data['TOL'] = pd.to_numeric(combined_games_data['TOL'], errors='coerce')

In [15]:
# Make sure data is in datatime
combined_games_data.index = pd.to_datetime(combined_games_data.index)

# Define the start and end dates for the 2024 NFL season 
season_start = '2024-09-01'  # Start of 2024 season
season_end = '2025-02-28'    # End of 2024 season

# Convert to datetime
season_start = pd.to_datetime(season_start)
season_end = pd.to_datetime(season_end)

# Filter out games from the 2024 season using the index
df_2024 = combined_games_data[(combined_games_data.index >= season_start) & (combined_games_data.index <= season_end)]

# Filter out all other games
combined_games_data = combined_games_data[(combined_games_data.index < season_start) | (combined_games_data.index > season_end)]

# Save the 2024 data to a CSV for future use
df_2024.to_csv('2024_season_games.csv')

In [16]:
#Rename df
NFL = combined_games_data

In [17]:
#Feature engineering for differentials (yards, turnovers, score)
#Points differential
NFL["PointDiff"] = NFL["PtsW"] - NFL ["PtsL"]

#Yard differential
NFL["YardDiff"] = NFL["YdsW"] - NFL["YdsL"]

#Turnover differential
NFL["TurnoverDiff"] = NFL["TOW"] - NFL["TOL"]

In [18]:
print(NFL.head())

           Week  Day    Time                 Winner Awaygame  \
Date                                                           
2023-09-07    1  Thu  8:20PM          Detroit Lions        @   
2023-09-10    1  Sun  1:00PM        Atlanta Falcons            
2023-09-10    1  Sun  1:00PM       Cleveland Browns            
2023-09-10    1  Sun  1:00PM   Jacksonville Jaguars        @   
2023-09-10    1  Sun  1:00PM  Washington Commanders            

                         Loser  PtsW  PtsL  YdsW  TOW  YdsL  TOL  Tie  \
Date                                                                    
2023-09-07  Kansas City Chiefs    21    20   368    1   316    1    0   
2023-09-10   Carolina Panthers    24    10   221    0   281    3    0   
2023-09-10  Cincinnati Bengals    24     3   350    2   142    0    0   
2023-09-10  Indianapolis Colts    31    21   342    2   280    3    0   
2023-09-10   Arizona Cardinals    20    16   248    3   210    2    0   

            PointDiff  YardDiff  Turnov

In [19]:
# Create the 'Home' and 'Away' columns based on the 'Awaygame' column
NFL['Home'] = NFL['Awaygame'].apply(lambda x: True if x == '' else False)
NFL['Away'] = NFL['Awaygame'].apply(lambda x: True if x == '@' else False)

# For neutral games (N), we can set both as False (or leave them as NaN if you prefer)
NFL.loc[NFL['Awaygame'] == 'N', ['Home', 'Away']] = False, False

# Check the new columns
NFL[['Awaygame', 'Home', 'Away']].head()

,Awaygame,Home,Away
Date,,,
2023-09-07,@,False,True
2023-09-10,,True,False
2023-09-10,,True,False
2023-09-10,@,False,True
2023-09-10,,True,False


In [20]:
#Check N values to make sure Home and Away columns are false
NFL[NFL["Awaygame"]=='N']

,Week,Day,Time,Winner,Awaygame,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Tie,PointDiff,YardDiff,TurnoverDiff,Home,Away
Date,,,,,,,,,,,,,,,,,,
2024-02-11,SuperBowl,Sun,6:30PM,Kansas City Chiefs,N,San Francisco 49ers,25,22,455,2,382,2,0,3,73,0,False,False
2023-02-12,SuperBowl,Sun,6:30PM,Kansas City Chiefs,N,Philadelphia Eagles,38,35,340,0,417,1,0,3,-77,-1,False,False
2022-02-13,SuperBowl,Sun,6:30PM,Los Angeles Rams,N,Cincinnati Bengals,23,20,313,2,305,0,0,3,8,2,False,False
2021-02-07,SuperBowl,Sun,6:30PM,Tampa Bay Buccaneers,N,Kansas City Chiefs,31,9,340,0,350,2,0,22,-10,-2,False,False
2020-02-02,SuperBowl,Sun,6:30PM,Kansas City Chiefs,N,San Francisco 49ers,31,20,397,2,351,2,0,11,46,0,False,False
2019-02-03,SuperBowl,Sun,6:30PM,New England Patriots,N,Los Angeles Rams,13,3,407,1,260,1,0,10,147,0,False,False
2018-02-04,SuperBowl,Sun,6:30PM,Philadelphia Eagles,N,New England Patriots,41,33,538,1,613,1,0,8,-75,0,False,False


In [21]:
#Looking at dataframe
NFL

,Week,Day,Time,Winner,Awaygame,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Tie,PointDiff,YardDiff,TurnoverDiff,Home,Away
Date,,,,,,,,,,,,,,,,,,
2023-09-07,1,Thu,8:20PM,Detroit Lions,@,Kansas City Chiefs,21,20,368,1,316,1,0,1,52,0,False,True
2023-09-10,1,Sun,1:00PM,Atlanta Falcons,,Carolina Panthers,24,10,221,0,281,3,0,14,-60,-3,True,False
2023-09-10,1,Sun,1:00PM,Cleveland Browns,,Cincinnati Bengals,24,3,350,2,142,0,0,21,208,2,True,False
2023-09-10,1,Sun,1:00PM,Jacksonville Jaguars,@,Indianapolis Colts,31,21,342,2,280,3,0,10,62,-1,False,True
2023-09-10,1,Sun,1:00PM,Washington Commanders,,Arizona Cardinals,20,16,248,3,210,2,0,4,38,1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-14,Division,Sun,1:05PM,Jacksonville Jaguars,@,Pittsburgh Steelers,45,42,378,0,545,2,0,3,-167,-2,False,True
2018-01-14,Division,Sun,4:40PM,Minnesota Vikings,,New Orleans Saints,29,24,403,1,358,2,0,5,45,-1,True,False
2018-01-21,ConfChamp,Sun,3:05PM,New England Patriots,,Jacksonville Jaguars,24,20,344,1,374,0,0,4,-30,1,True,False


In [22]:
NFL

,Week,Day,Time,Winner,Awaygame,Loser,PtsW,PtsL,YdsW,TOW,YdsL,TOL,Tie,PointDiff,YardDiff,TurnoverDiff,Home,Away
Date,,,,,,,,,,,,,,,,,,
2023-09-07,1,Thu,8:20PM,Detroit Lions,@,Kansas City Chiefs,21,20,368,1,316,1,0,1,52,0,False,True
2023-09-10,1,Sun,1:00PM,Atlanta Falcons,,Carolina Panthers,24,10,221,0,281,3,0,14,-60,-3,True,False
2023-09-10,1,Sun,1:00PM,Cleveland Browns,,Cincinnati Bengals,24,3,350,2,142,0,0,21,208,2,True,False
2023-09-10,1,Sun,1:00PM,Jacksonville Jaguars,@,Indianapolis Colts,31,21,342,2,280,3,0,10,62,-1,False,True
2023-09-10,1,Sun,1:00PM,Washington Commanders,,Arizona Cardinals,20,16,248,3,210,2,0,4,38,1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-14,Division,Sun,1:05PM,Jacksonville Jaguars,@,Pittsburgh Steelers,45,42,378,0,545,2,0,3,-167,-2,False,True
2018-01-14,Division,Sun,4:40PM,Minnesota Vikings,,New Orleans Saints,29,24,403,1,358,2,0,5,45,-1,True,False
2018-01-21,ConfChamp,Sun,3:05PM,New England Patriots,,Jacksonville Jaguars,24,20,344,1,374,0,0,4,-30,1,True,False


In [23]:
#Combine duplicate teams into one
# Define the mapping to combine the Washington team names and the Raiders team names
team_name_mapping = {
    'Washington Redskins': 'Washington Commanders',
    'Washington Football Team': 'Washington Commanders',
    'Washington Commanders': 'Washington Commanders',
    'Oakland Raiders': 'Las Vegas Raiders',
    'Las Vegas Raiders': 'Las Vegas Raiders'
}

# Replace the team names in both 'Winner' and 'Loser' columns
NFL['Winner'] = NFL['Winner'].replace(team_name_mapping)
NFL['Loser'] = NFL['Loser'].replace(team_name_mapping)

# Check that the replacement was successful
NFL[['Winner', 'Loser']].drop_duplicates().head()

,Winner,Loser
Date,,
2023-09-07,Detroit Lions,Kansas City Chiefs
2023-09-10,Atlanta Falcons,Carolina Panthers
2023-09-10,Cleveland Browns,Cincinnati Bengals
2023-09-10,Jacksonville Jaguars,Indianapolis Colts
2023-09-10,Washington Commanders,Arizona Cardinals


In [24]:
#Check for duplicate teams
print(NFL["Winner"].unique())

['Detroit Lions' 'Atlanta Falcons' 'Cleveland Browns'
 'Jacksonville Jaguars' 'Washington Commanders' 'Baltimore Ravens'
 'Tampa Bay Buccaneers' 'New Orleans Saints' 'San Francisco 49ers'
 'Green Bay Packers' 'Las Vegas Raiders' 'Miami Dolphins'
 'Philadelphia Eagles' 'Los Angeles Rams' 'Dallas Cowboys' 'New York Jets'
 'Buffalo Bills' 'Indianapolis Colts' 'Seattle Seahawks'
 'Kansas City Chiefs' 'Tennessee Titans' 'New York Giants'
 'Pittsburgh Steelers' 'Houston Texans' 'Los Angeles Chargers'
 'New England Patriots' 'Arizona Cardinals' 'Cincinnati Bengals'
 'Minnesota Vikings' 'Denver Broncos' 'Chicago Bears' 'Carolina Panthers']


Team Dataframe

In [25]:
# Make sure NFL DataFrame is sorted by Date (ascending or descending)
NFL.index = pd.to_datetime(NFL.index)  # Convert the index (Date) to datetime, if it's not already

# Sort the DataFrame by the Date index (ascending or descending)
NFL = NFL.sort_index(ascending=True)

In [26]:
# Initialize an empty DataFrame to hold the stats
team_stats = pd.DataFrame(columns=[
    'Total_Games', 'Win_Rate', 'Avg_PtsScored', 'Avg_PtsAllowed', 'Avg_Yards', 'Avg_YardsAllowed', 'Avg_TurnoverDiff', 
    'Avg_PointDiff', 'Avg_YardDiff',
    'Home_PtsScored', 'Away_PtsScored', 'Home_Yards', 'Away_Yards', 'Home_Turnover', 'Away_Turnover',
    'Last5_Win_Rate', 'Last5_Avg_PtsScored', 'Last5_Avg_PtsAllowed', 'Last5_Avg_TurnoverDiff', 'Last5_Avg_PointDiff'
])

# Iterate through each team
for team in NFL['Winner'].unique():
    # Total games for the team (count all games where the team was either winner or loser)
    total_games = len(NFL[(NFL['Winner'] == team) | (NFL['Loser'] == team)])  # Count both wins and losses

    # Calculate the win rate based on the number of wins and total games
    win_rate = len(NFL[(NFL['Winner'] == team) & (NFL['Tie'] == False)]) / total_games  # Wins divided by total games played

    # Stats where the team was the winner
    win_games = NFL[(NFL['Winner'] == team) & (NFL['Tie'] == False)]
    sum_pts_scored_win = win_games['PtsW'].sum()  # Sum points scored in wins
    sum_pts_allowed_win = win_games['PtsL'].sum()  # Sum points allowed in losses
    sum_yards_win = win_games['YdsW'].sum()  # Sum yards gained in wins
    sum_yards_allowed_win = win_games['YdsL'].sum()  # Sum yards allowed in losses
    sum_turnover_diff_win = win_games['TurnoverDiff'].sum()  # Sum turnover diff in wins
    sum_point_diff_win = win_games['PointDiff'].sum()  # Sum point diff in wins
    sum_yard_diff_win = win_games['YardDiff'].sum()  # Sum yard diff in wins

    # Stats where the team was the loser
    loss_games = NFL[(NFL['Loser'] == team) & (NFL['Tie'] == False)]
    sum_pts_scored_loss = loss_games['PtsL'].sum()  # Sum points scored in losses
    sum_pts_allowed_loss = loss_games['PtsW'].sum()  # Sum points allowed in losses
    sum_yards_loss = loss_games['YdsL'].sum()  # Sum yards scored in losses
    sum_yards_allowed_loss = loss_games['YdsW'].sum()  # Sum yards allowed in losses
    sum_turnover_diff_loss = (-loss_games['TurnoverDiff'].sum())  # Sum turnover diff in losses
    sum_point_diff_loss = (-loss_games['PointDiff'].sum())  # Sum point diff in losses
    sum_yard_diff_loss = (-loss_games['YardDiff'].sum())  # Sum yard diff in losses

    #Games where team was Home and Away
    home_games_team = NFL[((NFL['Home'] == True) & (NFL['Winner'] == team)) | ((NFL['Home'] == False) & (NFL['Loser'] == team))]
    away_games_team = NFL[((NFL['Away'] == True) & (NFL['Winner'] == team)) | ((NFL['Away'] == False) & (NFL['Loser'] == team))]

    # Home and Away stats for the team
    home_pts_scored = (
    pd.concat([
        home_games_team[home_games_team['Winner'] == team]['PtsW'],
        home_games_team[home_games_team['Loser'] == team]['PtsL']
    ]).mean() if len(home_games_team) > 0 else 0)
    away_pts_scored = (
    pd.concat([
        away_games_team[away_games_team['Winner'] == team]['PtsW'],
        away_games_team[away_games_team['Loser'] == team]['PtsL']
    ]).mean() if len(away_games_team) > 0 else 0)
    home_yards = (
    pd.concat([
        home_games_team[home_games_team['Winner'] == team]['YdsW'],
        home_games_team[home_games_team['Loser'] == team]['YdsL']
    ]).mean() if len(home_games_team) > 0 else 0)
    away_yards = (
    pd.concat([
        away_games_team[away_games_team['Winner'] == team]['YdsW'],
        away_games_team[away_games_team['Loser'] == team]['YdsL']
    ]).mean() if len(home_games_team) > 0 else 0)
    home_turnover =  (
    pd.concat([
        home_games_team[home_games_team['Winner'] == team]['TOW'],
        home_games_team[home_games_team['Loser'] == team]['TOL']
    ]).mean() if len(home_games_team) > 0 else 0)
    away_turnover =  (
    pd.concat([
        away_games_team[away_games_team['Winner'] == team]['TOW'],
        away_games_team[away_games_team['Loser'] == team]['TOL']
    ]).mean() if len(home_games_team) > 0 else 0)

    # Last 5 games stats
    last5_games = NFL[(NFL['Winner'] == team) | (NFL['Loser'] == team)].tail(5)
    last5_win_rate = len(last5_games[last5_games['Winner'] == team]) / len(last5_games)
    last5_pts_scored = last5_games['PtsW'].mean() if len(last5_games) > 0 else 0
    last5_pts_allowed = last5_games['PtsL'].mean() if len(last5_games) > 0 else 0
    last5_turnover_diff = last5_games['TurnoverDiff'].mean() if len(last5_games) > 0 else 0
    last5_point_diff = last5_games['PointDiff'].mean() if len(last5_games) > 0 else 0


    # Add all stats to the new dataframe
    team_stats.loc[team] = [
        total_games, win_rate, 
        (sum_pts_scored_win + sum_pts_scored_loss) / total_games,  # Average points scored across wins and losses
        (sum_pts_allowed_win + sum_pts_allowed_loss) / total_games,  # Average points allowed across wins and losses
        (sum_yards_win + sum_yards_loss) / total_games,  # Average yards across wins and losses
        (sum_yards_allowed_win + sum_yards_allowed_loss) / total_games,  # Average yards allowed across wins and losses
        (sum_turnover_diff_win + sum_turnover_diff_loss) / total_games,  # Average turnover diff across wins and losses
        (sum_point_diff_win + sum_point_diff_loss) / total_games,  # Average point diff across wins and losses
        (sum_yard_diff_win + sum_yard_diff_loss) / total_games,  # Average yard diff across wins and losses
        
        home_pts_scored, away_pts_scored, home_yards, away_yards, home_turnover, away_turnover,
        
        last5_win_rate, last5_pts_scored, last5_pts_allowed, last5_turnover_diff, last5_point_diff  # Last 5 games stats
    ]

# Show the new team stats dataframe
team_stats.head()



,Total_Games,Win_Rate,Avg_PtsScored,Avg_PtsAllowed,Avg_Yards,Avg_YardsAllowed,Avg_TurnoverDiff,Avg_PointDiff,Avg_YardDiff,Home_PtsScored,Away_PtsScored,Home_Yards,Away_Yards,Home_Turnover,Away_Turnover,Last5_Win_Rate,Last5_Avg_PtsScored,Last5_Avg_PtsAllowed,Last5_Avg_TurnoverDiff,Last5_Avg_PointDiff
Kansas City Chiefs,134.0,0.746269,28.440299,21.626866,394.067164,352.111940,-0.261194,6.813433,41.955224,27.069444,29.616667,377.763889,412.733333,1.000000,1.316667,1.0,21.6,15.0,-0.4,6.6
Dallas Cowboys,121.0,0.586777,26.165289,21.661157,373.115702,336.024793,-0.239669,4.504132,37.090909,30.016667,22.377049,396.683333,349.934426,1.250000,1.196721,0.4,31.8,18.2,-1.2,13.6
Green Bay Packers,122.0,0.590164,24.098361,22.188525,349.221311,337.344262,-0.229508,1.909836,11.877049,24.557377,24.114754,357.934426,346.262295,0.885246,1.163934,0.8,31.0,20.4,-0.8,10.6
Carolina Panthers,116.0,0.362069,20.189655,24.189655,322.603448,336.543103,0.327586,-4.000000,-13.939655,20.754386,19.644068,327.245614,318.118644,1.403509,1.525424,0.2,21.0,8.6,-1.2,12.4
Los Angeles Rams,126.0,0.611111,25.333333,21.753968,362.904762,333.896825,-0.087302,3.579365,29.007937,25.161290,25.187500,364.274194,360.750000,1.274194,1.312500,0.8,25.8,22.0,0.4,3.8


In [27]:
print(team_stats.describe())
print(team_stats.info())
print(team_stats.shape)

       Total_Games   Win_Rate  Avg_PtsScored  Avg_PtsAllowed   Avg_Yards  \
count    32.000000  32.000000      32.000000       32.000000   32.000000   
mean    120.250000   0.495140      22.642509       22.746176  342.537412   
std       4.212118   0.108563       2.540729        1.675091   22.142745   
min     115.000000   0.313043      17.600000       18.803279  293.052174   
25%     117.000000   0.411441      21.049313       21.730765  327.369453   
50%     120.000000   0.489352      22.413587       22.903319  345.247126   
75%     122.000000   0.587141      24.526488       24.213362  356.574460   
max     134.000000   0.746269      28.440299       25.457627  394.067164   

       Avg_YardsAllowed  Avg_TurnoverDiff  Avg_PointDiff  Avg_YardDiff  \
count         32.000000         32.000000      32.000000     32.000000   
mean         343.248704          0.003972      -0.103667     -0.711292   
std           14.813439          0.222540       3.767077     29.769338   
min          315.77

In [28]:
team_stats

,Total_Games,Win_Rate,Avg_PtsScored,Avg_PtsAllowed,Avg_Yards,Avg_YardsAllowed,Avg_TurnoverDiff,Avg_PointDiff,Avg_YardDiff,Home_PtsScored,Away_PtsScored,Home_Yards,Away_Yards,Home_Turnover,Away_Turnover,Last5_Win_Rate,Last5_Avg_PtsScored,Last5_Avg_PtsAllowed,Last5_Avg_TurnoverDiff,Last5_Avg_PointDiff
Kansas City Chiefs,134.0,0.746269,28.440299,21.626866,394.067164,352.111940,-0.261194,6.813433,41.955224,27.069444,29.616667,377.763889,412.733333,1.000000,1.316667,1.0,21.6,15.0,-0.4,6.6
Dallas Cowboys,121.0,0.586777,26.165289,21.661157,373.115702,336.024793,-0.239669,4.504132,37.090909,30.016667,22.377049,396.683333,349.934426,1.250000,1.196721,0.4,31.8,18.2,-1.2,13.6
Green Bay Packers,122.0,0.590164,24.098361,22.188525,349.221311,337.344262,-0.229508,1.909836,11.877049,24.557377,24.114754,357.934426,346.262295,0.885246,1.163934,0.8,31.0,20.4,-0.8,10.6
Carolina Panthers,116.0,0.362069,20.189655,24.189655,322.603448,336.543103,0.327586,-4.000000,-13.939655,20.754386,19.644068,327.245614,318.118644,1.403509,1.525424,0.2,21.0,8.6,-1.2,12.4
Los Angeles Rams,126.0,0.611111,25.333333,21.753968,362.904762,333.896825,-0.087302,3.579365,29.007937,25.161290,25.187500,364.274194,360.750000,1.274194,1.312500,0.8,25.8,22.0,0.4,3.8
Philadelphia Eagles,126.0,0.595238,24.880952,21.952381,357.769841,335.206349,0.079365,2.928571,22.563492,25.666667,24.365079,360.555556,359.063492,1.190476,1.492063,0.2,29.4,18.4,-0.8,11.0
Atlanta Falcons,117.0,0.427350,22.042735,23.905983,349.572650,357.401709,0.179487,-1.863248,-7.829060,23.543860,20.616667,363.684211,336.166667,1.298246,1.333333,0.2,30.4,15.2,-2.2,15.2
Jacksonville Jaguars,120.0,0.383333,20.233333,23.141667,334.433333,349.341667,0.258333,-2.908333,-14.908333,19.800000,20.666667,331.600000,337.266667,1.450000,1.633333,0.2,27.6,13.2,-1.6,14.4
Buffalo Bills,125.0,0.624000,24.376000,20.088000,356.176000,318.328000,-0.192000,4.288000,37.848000,25.338462,23.333333,352.307692,360.366667,1.261538,1.533333,0.8,26.0,19.6,0.0,6.4
Baltimore Ravens,122.0,0.631148,25.540984,18.803279,362.819672,315.778689,-0.188525,6.737705,47.040984,26.564516,24.483333,365.887097,359.650000,1.322581,1.116667,0.6,31.4,13.6,-2.0,17.8


In [29]:
# Rounding the columns based on the suggested decimal places for 'team_stats' DataFrame

team_stats['Win_Rate'] = team_stats['Win_Rate'].round(2)           # Round to 2 decimal places
team_stats['Avg_PtsScored'] = team_stats['Avg_PtsScored'].round(2) # Round to 2 decimal places
team_stats['Avg_PtsAllowed'] = team_stats['Avg_PtsAllowed'].round(2) # Round to 2 decimal places
team_stats['Avg_Yards'] = team_stats['Avg_Yards'].round(2)         # Round to 2 decimal places
team_stats['Avg_YardsAllowed'] = team_stats['Avg_YardsAllowed'].round(2) # Round to 2 decimal places
team_stats['Avg_TurnoverDiff'] = team_stats['Avg_TurnoverDiff'].round(2) # Round to 2 decimal places
team_stats['Avg_PointDiff'] = team_stats['Avg_PointDiff'].round(2) # Round to 2 decimal places
team_stats['Avg_YardDiff'] = team_stats['Avg_YardDiff'].round(2)   # Round to 2 decimal places
team_stats['Home_PtsScored'] = team_stats['Home_PtsScored'].round(2) # Round to 2 decimal places
team_stats['Away_PtsScored'] = team_stats['Away_PtsScored'].round(2) # Round to 2 decimal places
team_stats['Home_Yards'] = team_stats['Home_Yards'].round(2)       # Round to 2 decimal places
team_stats['Away_Yards'] = team_stats['Away_Yards'].round(2)       # Round to 2 decimal places
team_stats['Home_Turnover'] = team_stats['Home_Turnover'].round(2) # Round to 2 decimal places
team_stats['Away_Turnover'] = team_stats['Away_Turnover'].round(2) # Round to 2 decimal places

team_stats['Last5_Win_Rate'] = team_stats['Last5_Win_Rate'].round(2)
team_stats['Last5_Avg_PtsScored'] = team_stats['Last5_Avg_PtsScored'].round(2)
team_stats['Last5_Avg_PtsAllowed'] = team_stats['Last5_Avg_PtsAllowed'].round(2)
team_stats['Last5_Avg_TurnoverDiff'] = team_stats['Last5_Avg_TurnoverDiff'].round(2)
team_stats['Last5_Avg_PointDiff'] = team_stats['Last5_Avg_PointDiff'].round(2)


In [30]:
print(team_stats)

                       Total_Games  Win_Rate  Avg_PtsScored  Avg_PtsAllowed  \
Kansas City Chiefs           134.0      0.75          28.44           21.63   
Dallas Cowboys               121.0      0.59          26.17           21.66   
Green Bay Packers            122.0      0.59          24.10           22.19   
Carolina Panthers            116.0      0.36          20.19           24.19   
Los Angeles Rams             126.0      0.61          25.33           21.75   
Philadelphia Eagles          126.0      0.60          24.88           21.95   
Atlanta Falcons              117.0      0.43          22.04           23.91   
Jacksonville Jaguars         120.0      0.38          20.23           23.14   
Buffalo Bills                125.0      0.62          24.38           20.09   
Baltimore Ravens             122.0      0.63          25.54           18.80   
Las Vegas Raiders            116.0      0.42          21.19           25.44   
Detroit Lions                118.0      0.42        

In [31]:
#Save Data as CSV
team_stats.to_csv('Team_Stats.csv', index=True)
NFL.to_csv('NFL.csv', index=True)